**Importing Libraries and Data**

In [ ]:
#Importing libraries
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import root_mean_squared_error
from statsmodels.tsa.stattools import acf
import matplotlib.pyplot as plt
import prophet
import numpy as np
import warnings
import gc
import os
import sys

#Ignoring warnings
warnings.filterwarnings("ignore")

#Reading in the data and extracting unique account values
BoostedTransactions = pd.read_csv("Longer Subset of Transaction Data (14 Months).csv")
AccountIDs = BoostedTransactions["AccountId"].values
UniqueAccounts = set(AccountIDs)

#Converting transaction dates into months
BoostedTransactions["TransactionDate"] = pd.to_datetime(BoostedTransactions["TransactionDate"])
BoostedTransactions["TransactionDate"] = BoostedTransactions["TransactionDate"].dt.to_period("M").dt.to_timestamp()

#Grouping transactions by account and month and summating each grouping's net transaction amount
GroupedSumOfTransactionsByAccountAndMonth = BoostedTransactions.groupby(by=["AccountId", "TransactionDate"]).agg(["sum"])["Amount"]
print(GroupedSumOfTransactionsByAccountAndMonth)
GroupedSumOfTransactionsByAccountAndMonth = GroupedSumOfTransactionsByAccountAndMonth.reset_index()

                                                          sum
AccountId                            TransactionDate         
0003a5ae-0c77-4372-b44d-882ef9874a28 2019-05-01      -5541.31
                                     2019-06-01      -3601.46
                                     2019-07-01       3151.74
                                     2019-08-01      -4801.13
                                     2019-09-01       3890.05
...                                                       ...
fff7f00c-c869-4310-b705-4503538f5ecf 2020-03-01       2974.84
                                     2020-04-01        382.93
                                     2020-05-01        166.15
                                     2020-06-01       -550.89
                                     2020-07-01      -2721.18

[7632 rows x 1 columns]


**Key Prophet Loop**

In [ ]:
#Creating models and forecasting for all accounts
for i in UniqueAccounts:
    print(i)
    try:
        #Selecting only the selected amount's time series and ensuring it has no gaps
        ForecastingTimeSeries = GroupedSumOfTransactionsByAccountAndMonth[GroupedSumOfTransactionsByAccountAndMonth["AccountId"] == i][["TransactionDate", "sum"]]

        ForecastingTimeSeries = ForecastingTimeSeries.set_index("TransactionDate")

        ForecastingTimeSeries = ForecastingTimeSeries.resample("MS").sum()

        ForecastingTimeSeries = ForecastingTimeSeries.reset_index()

        #Calculating the cutoff point between training and testing sets
        TrainEnd = int(0.7 * len(ForecastingTimeSeries))

        #Splitting the data into training and testing sets and adjusting column names in each
        TrainData = ForecastingTimeSeries[:TrainEnd]
        TestData = ForecastingTimeSeries[TrainEnd:]

        TrainData.columns = ["ds", "y"]
        TestData.columns = ["ds", "y"]

        #Declaring and training model
        Model = prophet.Prophet()
        Model.fit(TrainData)

        #Performing a test forecast
        Forecast = Model.predict(TestData)

        #Calculating error rates and appending them to the errors CSV
        try:
          RMSE = root_mean_squared_error(Forecast["yhat"], TestData["y"])
          SI = abs(RMSE/abs(TestData["y"]).mean())
          AccountError = pd.DataFrame({"AccountID" : [i],
                                        "RMSE" : [RMSE],
                                        "SI" : [SI]})
          AccountError.to_csv("Prophet Longer Subset Errors (Month-by-Month).csv", mode='a', header=not os.path.exists("Prophet Longer Subset Errors (Month-by-Month).csv"), index=False)
        #Where a ValueError occurs, the loop is broken
        except ValueError:
          print("ValueError encountered")
          break

        #Forecasting the next 12 months
        Future = Model.make_future_dataframe(periods=12, freq="MS")
        FutureForecast = Model.predict(Future)

        FutureForecastTail = FutureForecast.tail(12)
        FutureForecastValues = FutureForecastTail["yhat"].values

        #Calculating highest amount, lowest amount and difference and appending them into the forecasts CSV
        HighestAmount = FutureForecastValues.max()
        LowestAmount = FutureForecastValues.min()

        if np.isnan(HighestAmount) or np.isnan(LowestAmount):
          HighestAmount = 0
          LowestAmount = 0
        else:
          pass

        Difference = HighestAmount - LowestAmount

        ForecastData = pd.DataFrame({"AccountID" : [i],
                                     "Highest Amount" : [HighestAmount],
                                     "Lowest Amount" : [LowestAmount],
                                     "Difference" : [Difference]})

        ForecastData.to_csv("Prophet Longer Subset Forecasts (Month-by-Month).csv", mode='a', header=not os.path.exists("Prophet Longer Subset Forecasts (Month-by-Month).csv"), index=False)

        print("Processed account number",i)
    #Where an error occurs, the user is informed about it
    except Exception as e:
        print(f"Error encountered processing account number {i}.")
        ErrorType, ErrorObject, ErrorTraceback = sys.exc_info()

        ErrorFilename = os.path.split(
            ErrorTraceback.tb_frame.f_code.co_filename
        )[1]

        ErrorMessage = str(e)

        ErrorLineNumber = ErrorTraceback.tb_lineno

        print(f'Exception Type: {ErrorType}')

        print(f'Exception Filename: {ErrorFilename}')

        print(f'Exception Line Number: {ErrorLineNumber}')

        print(f'Exception Message: {ErrorMessage}')
        break
    #Deleting model data to conserve RAM
    finally:
        del Model, Forecast, TrainData, TestData, ForecastingTimeSeries
        gc.collect()


1bd930ef-8d95-490b-8244-593fffa76943


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnno2kn4y/as1mp8yh.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnno2kn4y/1h33adn_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=56931', 'data', 'file=/tmp/tmpnno2kn4y/as1mp8yh.json', 'init=/tmp/tmpnno2kn4y/1h33adn_.json', 'output', 'file=/tmp/tmpnno2kn4y/prophet_modelruaq12fg/prophet_model-20250807125731.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
12:57:31 - cmdstanpy - INFO - Chain [1] start processing
IN

Processed account number 1bd930ef-8d95-490b-8244-593fffa76943
b398f850-a27e-49e3-8cb6-f350432095d8


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnno2kn4y/py76ablp.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnno2kn4y/hjfeb4qw.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=5383', 'data', 'file=/tmp/tmpnno2kn4y/py76ablp.json', 'init=/tmp/tmpnno2kn4y/hjfeb4qw.json', 'output', 'file=/tmp/tmpnno2kn4y/prophet_modelz2cns0xw/prophet_model-20250807125732.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
12:57:32 - cmdstanpy - INFO - Chain [1] start processing
INF

Processed account number b398f850-a27e-49e3-8cb6-f350432095d8
0475fbc0-729a-4529-9222-59b4b27408fe
Processed account number 0475fbc0-729a-4529-9222-59b4b27408fe


...

Processed account number 6cceb62f-8756-4950-b2b5-d1b7d5c37732
d994a8e3-00d1-4560-a25e-ba93583f96d4


13:00:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnno2kn4y/rwdmou_a.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnno2kn4y/a73z1jo8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=48302', 'data', 'file=/tmp/tmpnno2kn4y/rwdmou_a.json', 'init=/tmp/tmpnno2kn4y/a73z1jo8.json', 'output', 'file=/tmp/tmpnno2kn4y/prophet_modelwgpgjv6n/prophet_model-20250807130006.csv', 'method=optimiz

Processed account number d994a8e3-00d1-4560-a25e-ba93583f96d4
97e91e31-ff49-4714-99a5-8f09cdd13a67


13:00:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Processed account number 97e91e31-ff49-4714-99a5-8f09cdd13a67


**Calculating Error Statistics**

In [ ]:
#Calculating averaged error statistics
Errors = pd.read_csv("Prophet Longer Subset Errors (Month-by-Month).csv")
MeanRMSE = Errors["RMSE"].mean()
print("Mean RMSE:",MeanRMSE)
MedianRMSE = Errors["RMSE"].median()
print("Median RMSE:",MedianRMSE)
MeanSI = Errors["SI"].mean()
print("Mean SI:",MeanSI)
MedianSI = Errors["SI"].median()
print("Median SI:",MedianSI)

Mean RMSE: 40962.95222282307
Median RMSE: 6850.322276535559
Mean SI: 2.1165481737141127
Median SI: 1.5612630081063128
